In [1]:
import json
from datetime import datetime

def extract_messages(data):
    """
    从JSON数据中提取纯文本消息
    """
    messages = []

    if isinstance(data, dict):  # 如果是单条消息
        data = [data]
    
    for entry in data:
        if "text" in entry:
            text_field = entry["text"]
            if isinstance(text_field, list):  # 如果是列表
                text = "".join(
                    part["text"] if isinstance(part, dict) else part
                    for part in text_field
                )
            elif isinstance(text_field, str):  # 如果是纯字符串
                text = text_field
            else:
                text = ""
            messages.append({
                "text": text,
                "time": datetime.fromisoformat(entry["date"]),
                "sender": entry.get("from", "<unknown>")
            })
    
    return messages


In [ ]:
# 加载JSON数据
data = json.loads(open('result.json').read())["messages"]
# 提取消息文本
messages = extract_messages(data)
for i, message in enumerate(messages, 1):
    print(f"消息 {i}: {message}")

In [3]:
import re
def match_pattern(text, pattern):
  # 匹配正则表达式
  match = re.match(pattern, text)
  if match:
      extracted_data = match.groupdict()  # 获取匹配的命名组
      print(f"ID: {extracted_data['id']}")
      print(f"金额: {extracted_data['amount']} EUR")
      print(f"项目: {extracted_data['project']}")
      
      return {
            "id": extracted_data["id"],
            "amount": extracted_data["amount"],
            "project": extracted_data["project"],
      }
  else:
      print("未匹配到数据")
      
      return {}

In [ ]:
pattern = r"(?P<id>.+?) spent (?P<amount>\d+(\.\d+)?) EUR on (?P<project>.+)"

records = [{**match_pattern(message["text"], pattern), **message} for message in messages] 

records = [record for record in records if record.get("amount")] 

In [ ]:
records

In [6]:
import pandas as pd

# 转换为 DataFrame
df = pd.DataFrame(records)



In [ ]:
df

In [8]:
# 将 amount 转为浮点数类型
df['amount'] = df['amount'].astype(float)

# 统计每个 id 的总金额
summary = df.groupby('id')['amount'].sum().reset_index()
summary.rename(columns={'amount': 'total_amount'}, inplace=True)

summary.to_excel('summary.xlsx', index=False)



In [ ]:
df[["id", "amount", "project", "time"]].to_excel('records.xlsx', index=False)

In [10]:
# df[["id", "amount", "project", "time"]].to_json('records.json', index=False)
# convert time to yyyy-mm-dd hh:mm:ss

df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

df[["id", "amount", "project", "time"]].to_json('records.json', index=False)